## Sistema de Análise de Sentimentos

In [2]:
# Importando bibliotecas

import nltk, re, string
from nltk.corpus import stopwords, twitter_samples
import numpy as np
import pickle

In [3]:
# Preprocessamento dos tweets

def process_tweet(tweet):
    stemmer = nltk.PorterStemmer()
    stopwords_english = stopwords.words('english')
    tweet = re.sub(r'\$\w*', '', tweet)
    tweet = re.sub(r'^RT[\s]+', '', tweet)
    tweet = re.sub(r'https?:\/\/.*[\r\n]*', '', tweet)
    tweet = re.sub(r'#', '', tweet)
    tokenizer = nltk.TweetTokenizer(preserve_case=False, strip_handles=True, reduce_len=True)
    tweet_tokens = tokenizer.tokenize(tweet)

    tweets_clean = []
    for word in tweet_tokens:
        if (word not in stopwords_english and
                word not in string.punctuation):
            stem_word = stemmer.stem(word)  # stemming word
            tweets_clean.append(stem_word)

    return tweets_clean


In [4]:
# Esta é a parte mais importante de todo o código:
# A razão é que nosso conjunto de features, no qual treinaremos nosso modelo, será construído aqui.

def build_freqs(tweets, ys):
    """Construir frequências.
    Entrada:
        tweets: uma lista de tweets
        ys: uma matriz m x 1 com o rótulo de sentimento de cada tweet
            (0 ou 1)
    Saída:
        freqs: um dicionário que mapeia cada par (palavra, sentimento) para sua
        frequência
    """
    # Converter o array np em lista, pois o zip requer um iterável.
    # O squeeze é necessário, senão a lista terá um elemento.
    # Isso é um NOP se 'ys' já for uma lista.
    yslist = np.squeeze(ys).tolist()

    # Começar com um dicionário vazio e preenchê-lo percorrendo todos os tweets
    # e todas as palavras processadas em cada tweet.
    freqs = {}
    for y, tweet in zip(yslist, tweets):
        for word in process_tweet(tweet):
            par = (word, y)
            if par in freqs:
                freqs[par] += 1
            else:
                freqs[par] = 1

    return freqs


In [5]:
# Check de como o código funciona

tweets = ['i am happy', 'i am tricked', 'i am sad', 'i am tired', 'i am tired']
ys = [1, 0, 0, 0, 0]
res = build_freqs(tweets, ys)
print(res)

{('happi', 1): 1, ('trick', 0): 1, ('sad', 0): 1, ('tire', 0): 2}


In [6]:
nltk.download('twitter_samples')
nltk.download('stopwords')

[nltk_data] Downloading package twitter_samples to C:\Users\Sepp-Kali-
[nltk_data]     Linux\AppData\Roaming\nltk_data...
[nltk_data]   Package twitter_samples is already up-to-date!
[nltk_data] Downloading package stopwords to C:\Users\Sepp-Kali-
[nltk_data]     Linux\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [7]:
# seleção dos conjuntos de tweets
all_positive_tweets = twitter_samples.strings('positive_tweets.json')
all_negative_tweets = twitter_samples.strings('negative_tweets.json')

In [8]:
# separando os dados em duas partes: treino e teste(validação)
test_pos = all_positive_tweets[4000:]
train_pos = all_positive_tweets[:4000]
test_neg = all_negative_tweets[4000:]
train_neg = all_negative_tweets[:4000]

In [9]:
train_x = train_pos + train_neg
test_x = test_pos + test_neg

In [10]:
# Combinação de rótulos positivos e negativos
# Estamos construíndo nossa variável 'y'-meta aqui

train_y = np.append(np.ones((len(train_pos), 1)), np.zeros((len(train_neg), 1)), axis=0)
test_y = np.append(np.ones((len(test_pos), 1)), np.zeros((len(test_neg), 1)), axis=0)

In [11]:
# Criação do dicionário de frequência
freqs = build_freqs(train_x, train_y)

In [12]:
# Verificação de saída

print("type(freqs) = " + str(type(freqs)))
print("len(freqs) = " + str(len(freqs.keys())))

type(freqs) = <class 'dict'>
len(freqs) = 11337


In [13]:
# Teste de função

print('Exemplo de um tweet positivo: \n', train_x[24])
print('\nExemplo de versão processada do tweet: \n', process_tweet(train_x[24]))

Exemplo de um tweet positivo: 
 💅🏽💋 - :)))) haven't seen you in years

Exemplo de versão processada do tweet: 
 ['💅🏽', '💋', ':)', 'seen', 'year']


In [14]:
# Construção do modelo de Regressão Logística a partir do ZERO

# Função Sigmoide
def sigmoid(z):
    """
    Entrada:
        z: é a entrada (pode ser um escalar ou um array)
    Saída:
        h: a sigmoide de z
    """
    zz = np.negative(z)
    h = 1 / (1 + np.exp(zz))
    return h


In [15]:
# Função de Descida de Gradiente e Custo

def descidaGradiente(x, y, theta, alpha, num_iters):
    """
    Entrada:
        x: matriz de features, que é (m,n+1)
        y: rótulos correspondentes da matriz de entrada x, dimensões (m,1)
        theta: vetor de pesos, dimensão (n+1,1)
        alpha: taxa de aprendizado
        num_iters: número de iterações para treinar o modelo
    Saída:
        J: o custo final
        theta: seu vetor de pesos final
    Dica: você pode imprimir o custo para garantir que esteja diminuindo.
    """
    # obter 'm', o número de linhas na matriz x
    m = x.shape[0]
    for i in range(0, num_iters):
        z = np.dot(x, theta)
        h = sigmoid(z)
        # calcular a função de custo
        custo = -1. / m * (np.dot(y.transpose(), np.log(h)) + np.dot((1 - y).transpose(), np.log(1 - h)))
        # atualizar os pesos theta
        theta = theta - (alpha / m) * np.dot(x.transpose(), (h - y))

    custo = float(custo)
    return custo, theta


In [16]:
#  Extraindo os recursos

def extract_features(tweet, freqs):
    """
    Entrada:
        tweet: uma lista de palavras para um tweet
        freqs: um dicionário correspondente às frequências de cada tupla (palavra, rótulo)
    Saída:
        x: um vetor de características de dimensão (1,3)
    """

    word_l = process_tweet(tweet)
    x = np.zeros((1, 3))

    # o termo de viés é definido como 1
    x[0, 0] = 1

    for word in word_l:
        # incrementar a contagem de palavras para o rótulo positivo 1
        x[0, 1] += freqs.get((word, 1.0), 0)
        # incrementar a contagem de palavras para o rótulo negativo 0
        x[0, 2] += freqs.get((word, 0.0), 0)

    assert (x.shape == (1, 3))
    return x

In [17]:
# dados de treino e teste (validação)

tmp1 = extract_features(train_x[22], freqs)
print(tmp1)

[[1.000e+00 3.006e+03 1.240e+02]]


In [18]:
# Esses três números são o conjunto de recursos que construímos usando as funções build_freq() e extract_features().
# build_freq() constrói um dicionário tendo palavras como chaves e o número de vezes que elas ocorreram no corpus como valores.
# O recurso de extração leva em conta esses valores para palavras positivas e negativas, ou seja, tmp1[1] e tmp[2]


### Como esses recursos serão usados para previsões na regressão logística:

- Primeiro é construída uma hipótese que para o nosso caso será h(x) = b1 + b2*x1 + b3*x2
- aqui b1 = 1, b2 e b3 são determinados pela função de custo e gradiente, x1 e x2 são o conjunto de recursos de palavras positivas e negativas.

In [20]:
# Treinamento do modelo

# coleta as características 'x' e as empilha em uma matriz 'X'
X = np.zeros((len(train_x), 3))
for i in range(len(train_x)):
    X[i, :] = extract_features(train_x[i], freqs)

# rótulos de treinamento correspondentes a X
Y = train_y

# Aplica a descida de gradiente
# esses valores são predefinido
J, theta = descidaGradiente(X, Y, np.zeros((3, 1)), 1e-9, 1500)

In [21]:
def prever_tweet(tweet, freqs, theta):
    """
    Entrada:
        tweet: uma string
        freqs: um dicionário correspondente às frequências de cada tupla (palavra, rótulo)
        theta: vetor de pesos (3,1)
    Saída:
        y_pred: a probabilidade de um tweet ser positivo ou negativo
    """
    # extrai as características do tweet e armazena em x
    x = extract_features(tweet, freqs)
    y_pred = sigmoid(np.dot(x, theta))

    return y_pred



In [23]:
def testar_regressao_logistica(test_x, test_y, freqs, theta):
    """
    Entrada:
        test_x: uma lista de tweets
        test_y: vetor (m, 1) com os rótulos correspondentes para a lista de tweets
        freqs: um dicionário com a frequência de cada par (ou tupla)
        theta: vetor de pesos de dimensão (3, 1)
    Saída:
        precisao: (# de tweets classificados corretamente) / (# total de tweets)
    """
    # lista para armazenar as previsões
    y_hat = []

    for tweet in test_x:
        # obter a previsão de rótulo para o tweet
        y_pred = prever_tweet(tweet, freqs, theta)
        if y_pred > 0.5:
            y_hat.append(1)
        else:
            y_hat.append(0)

    accuracy = (y_hat == np.squeeze(test_y)).sum() / len(test_x)

    return accuracy


In [24]:
tmp_accuracy = testar_regressao_logistica(test_x, test_y, freqs, theta)
print(f"Precisão do modelo de Regressão Logística = {tmp_accuracy:.4f}")

Precisão do modelo de Regressão Logística = 0.9950


In [25]:
# Fazendo 'Predict' com o próprio Tweet para teste

def pre(sentence):
    yhat = prever_tweet(sentence, freqs, theta)
    if yhat > 0.5:
        return 'Sentimento: Positivo'
    elif yhat == 0:
        return 'Sentimento: Neutro'
    else:
        return 'Sentimento: Negativo'

In [33]:
my_tweet = 'I love this song'

res = pre(my_tweet)
print(res)

Sentimento: Positivo


In [32]:
my_tweet = 'I am sad'

res = pre(my_tweet)
print(res)

Sentimento: Negativo
